In [1]:
# THIS IS A BAD EXAMPLE LOOK INTO IT!

'/businessindustryandtrade/changestobusiness/mergersandacquisitions/datasets/mergersandacquisitionsinvolvingukcompanies'

'/businessindustryandtrade/changestobusiness/mergersandacquisitions/datasets/mergersandacquisitionsinvolvingukcompanies'

In [1]:
import torch
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
import spacy
nlp = spacy.load('en_core_web_trf')

from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(2,3))

import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

In [3]:
from helpers.cloze_generation import generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.table_processing import preprocess_table, read_process_table, find_relevant_column_header, find_relevant_content
from helpers.t5_language_model import summarise_t5_results


df = pd.read_pickle('pickles/dataset_20210625_184837.pkl')
clozes_df = pd.read_json('pickles/clozes_20210807_165700.json')

In [4]:
ALL_DATA = os.listdir('datasets/')
ALL_DATA = [x[:-4] for x in ALL_DATA]

In [5]:
from xlrd import XLRDError


usability = []
for entry in tqdm(ALL_DATA):
    path = 'datasets/' + entry + '.xls'
    try:
        excel_file = pd.ExcelFile(path)
        
        sheet_names = excel_file.sheet_names
        for sheet_counter, sheet_name in enumerate(sheet_names):

            try:
                df = excel_file.parse(sheet_name)
                cleaned_df = preprocess_table(df)

                N, D = cleaned_df.shape
                usability.append(
                    {'rows': N, 'columns': D, 
                    'path': entry, 'sheet_name_counter': sheet_counter})
            except (ValueError, AttributeError, FileNotFoundError, KeyError, XLRDError):
                usability.append(
                    {'rows': -1, 'columns': -1, 
                    'path': entry, 'sheet_name_counter': sheet_counter})
    
    except (ValueError, AttributeError, FileNotFoundError, KeyError, XLRDError):
        usability.append(
                {'rows': -1, 'columns': -1, 
                 'path': entry, 'sheet_name_counter': -1})

  0%|          | 0/959 [00:00<?, ?it/s]/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/pafi

In [43]:
usable_frames = pd.DataFrame(usability)
usable_frames['usable'] = 0
usable_frames.loc[(
    ((usable_frames['rows'] > 5) & (usable_frames['rows'] < 250)) & ((usable_frames['columns'] > 5) & (usable_frames['columns'] < 250))
    ), 'usable'] = 1

In [45]:
pd.DataFrame(usable_frames).to_json('secondary/dataframe_usability_3.json')

In [44]:
usable_frames

,rows,columns,path,sheet_name_counter,usable
0,-1,-1,peoplepopulationandcommunity_birthsdeathsandma...,0,0
1,38,3,peoplepopulationandcommunity_birthsdeathsandma...,1,1
2,-1,-1,peoplepopulationandcommunity_birthsdeathsandma...,2,0
3,39,37,peoplepopulationandcommunity_birthsdeathsandma...,3,1
4,24,18,peoplepopulationandcommunity_birthsdeathsandma...,4,1
...,...,...,...,...,...
13218,34,82,economy_governmentpublicsectorandtaxes_publics...,17,1
13219,46,17,economy_governmentpublicsectorandtaxes_publics...,18,1
13220,46,10,economy_governmentpublicsectorandtaxes_publics...,19,1
13221,54,22,economy_governmentpublicsectorandtaxes_publics...,20,1


In [5]:
# usability = []
# for entry in tqdm(ALL_DATA):
    
#     try:
#         df_iterator = read_process_table(entry, file_ext='.xls')
        
#         done = False
        
#         df = next(df_iterator, False)
#         sheet_name_counter = 0

#         if df is None:
#             df = False

#         while df is not False:
#             N, D = df.shape
#             usability.append({'rows': N, 'columns': D, 'path': entry, 'sheet_name_counter': sheet_name_counter})
            
#             df = next(df_iterator, False)
#             sheet_name_counter += 1
#             # there are some excel files that have empty sheets...
#             # only way I managed to stop it 
#             if df is None: 
#                 df = False
    
#     except (ValueError, KeyError, AttributeError, FileNotFoundError):
#         usability.append(
#             {'rows': -1, 'columns': -1, 'path': entry, 'sheet_name_counter': -1})
#         continue


  1%|          | 10/958 [00:00<01:05, 14.36it/s]/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for Information. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for Contents. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for North East. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for North West. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specificati

In [8]:
pd.DataFrame(usability).to_json('secondary/dataframe_usability.json')

In [45]:
from copy import copy 

usability = pd.read_json('secondary/dataframe_usability.json')
print(usability.shape)
print(usability.describe())

print(usability[usability.rows == 124428].path.values)
#remove those that are unusable
usability = usability[usability.rows > 0] 
usability = usability[usability['columns'] > 0]
# remove those that are too large
usability = usability[usability.rows < 100] #remove those that are unusable
usability = usability[usability['columns'] < 10]

print(usability.shape)
print(usability.describe())

(2178, 4)
                rows      columns  sheet_name_counter
count    2178.000000  2178.000000         2178.000000
mean      223.668962    20.834711           26.504591
std      2887.228302    35.251094           46.738458
min        -1.000000    -1.000000           -1.000000
25%        22.000000     7.000000            2.000000
50%        47.000000    10.000000            7.000000
75%        90.000000    22.000000           22.000000
max    124428.000000   294.000000          233.000000
['/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsrelatedtodrugpoisoningbyselectedsubstances']
(828, 4)
             rows     columns  sheet_name_counter
count  828.000000  828.000000          828.000000
mean    38.549517    5.770531           22.026570
std     30.655731    2.002538           38.988037
min      3.000000    3.000000            0.000000
25%     14.000000    4.000000            1.000000
50%     25.000000    6.000000            5.500000
75%     55.000000    

In [46]:
entry = '/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsrelatedtodrugpoisoningbyselectedsubstances'
df_iterator = read_process_table(entry)

In [53]:
sav = next(df_iterator)
sav

,All drug poisonings,-,-,2020 Suicides Persons,2020 Suicides Males,2020 Suicides Females,Non-suicides Persons,Non-suicides Males,Non-suicides Females,2019 Suicides Persons,...,2019 Suicides Females,Non-suicides Persons,Non-suicides Males,Non-suicides Females,2018 Suicides Persons,2018 Suicides Males,2018 Suicides Females,Non-suicides Persons,Non-suicides Males,Number of deaths Females
0,All drug poisonings,NaN,NaN,870,475,395,3691,2633,1058,941,...,416,3452,2443,1009,906,488,418,3453,2496,957
1,of which: Drug misuse,NaN,NaN,363,211,152,2633,1954,679,413,...,152,2470,1822,648,418,255,163,2499,1870,629
2,Any opiate,"Any opiate (includes unspecified opiates, excl...",NaN,280,161,119,1983,1432,551,332,...,122,1828,1314,514,320,194,126,1888,1385,503
3,Any opiate,Heroin and Morphine,NaN,107,60,47,1230,921,309,152,...,55,1177,873,304,126,74,52,1210,940,270
4,Any opiate,Methadone,NaN,24,18,6,492,360,132,11,...,2,396,290,106,15,10,5,404,293,111
5,Any opiate,Tramadol,NaN,56,35,21,147,87,60,68,...,24,133,77,56,84,52,32,136,83,53
6,Any opiate,Codeine not from compound formulation,NaN,73,39,34,139,94,45,58,...,26,109,71,38,55,28,27,114,65,49
7,Any opiate,Dihydrocodeine not from compound formulation,NaN,17,6,11,79,52,27,22,...,6,80,48,32,15,8,7,82,51,31
8,Any opiate,Oxycodone,NaN,30,15,15,72,47,25,29,...,12,66,38,28,23,18,5,56,25,31
9,Any opiate,Fentanyl,NaN,10,7,3,47,33,14,14,...,4,45,29,16,12,8,4,62,39,23
